In [1]:
from readlif.reader import LifFile
import os
import numpy as np
from tifffile import imwrite
import napari

### Patways

In [2]:
file_path  = r'R:\Aidan\ULA timecourse_ARC_004.lif'

dir_save = r'R:\Aidan\test'

# choose bit depth for saving the tiff files
BIT_DEPTH = np.uint16

# choose True if you want to create small movies for each channel
CREATE_SMALL_MOVIES = True
SMALL_FACTOR = 4

### Look inside the file

In [3]:
lif_file = LifFile(file_path)

im_list = [(ind,x['name'],x['dims'], x['channels']) for ind,x in enumerate(lif_file.image_list)]
im_list

[(0, 'Image1/B2 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (1, 'Image1/B3 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (2, 'Image1/B4 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (3, 'Image1/C2 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (4, 'Image1/C3 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (5, 'Image1/C4 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (6, 'Image1/D2 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (7, 'Image1/D3 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (8, 'Image1/D4 Region1', Dims(x=2048, y=2048, z=9, t=11, m=21), 2),
 (9, 'Image1/B2 Region1_Merged', Dims(x=9614, y=9820, z=9, t=11, m=1), 2),
 (10, 'Image1/B3 Region1_Merged', Dims(x=9449, y=10290, z=9, t=11, m=1), 2),
 (11, 'Image1/B4 Region1_Merged', Dims(x=9726, y=9483, z=9, t=11, m=1), 2),
 (12, 'Image1/C2 Region1_Merged', Dims(x=9888, y=9586, z=9, t=11, m=1), 2),
 (13, 'Image1/C3 Region1_Merged', Dims(x=9322, y=10002, z=9, t=11, m=1), 2)

### Specify files to process

In [12]:
#sel_ind = range(len(im_list)) # for all available images
sel_ind = [0] # for selected images

im_list_sel = [lif_file.get_image(ind) for ind in sel_ind]
im_list_sel

['LifImage object with dimensions: Dims(x=2048, y=2048, z=9, t=11, m=21)']

### Process images

In [13]:
for im in im_list_sel:

    print(f'Processing image: {im.name}')

    for ch in range(im.channels):

        print(f'Processing channel: {ch}')

        # that has to be recalculated because m and t are swapped
        t_search = 0
        m_search = 0

        for t in range (im.dims.t):

            print(f'Processing frame: {t}')


            frame_projections_list = []
            frame_projection = np.zeros((im.dims.y,im.dims.x),dtype=BIT_DEPTH)
            frame_projections_list.append(frame_projection)

            additional_frames = [2,17,20]
        
            for m in range(im.dims.m):

                print(m_search)
                print(t_search % im.dims.t)

                frame_generator = im.get_iter_z(t = (t_search % im.dims.t), c=ch, m=m_search)

                t_search = t_search + 1
                m_search = t_search // im.dims.t

                frame_stack = np.zeros((im.dims.z,im.dims.y,im.dims.x),dtype=BIT_DEPTH)
                z = 0
                for frame in frame_generator:
                
                    frame_stack[z,:,:] = np.array(frame)
                    z += 1

                frame_projection = np.max(frame_stack,axis=0)

                frame_projections_list.append(frame_projection)

                if m in additional_frames:
                    frame_projection = np.zeros((im.dims.y,im.dims.x),dtype=BIT_DEPTH)
                    frame_projections_list.append(frame_projection)           


            for ind,frame in enumerate(frame_projections_list):

                # save the movie
                save_path = os.path.join(dir_save,im.name.split('/')[1] + '_ch_' + str(ch).zfill(2) + '_m_' + str(ind).zfill(2) + '_t_' + str(t).zfill(3) + '.tif')
                imwrite(save_path, frame, imagej=True)

Processing image: Image1/B2 Region1
Processing channel: 0
Processing frame: 0
Processing frame: 1
Processing frame: 2
Processing frame: 3
Processing frame: 4
Processing frame: 5
Processing frame: 6
Processing frame: 7
Processing frame: 8
Processing frame: 9
Processing frame: 10
Processing channel: 1
Processing frame: 0
Processing frame: 1
Processing frame: 2
Processing frame: 3
Processing frame: 4
Processing frame: 5
Processing frame: 6
Processing frame: 7
Processing frame: 8
Processing frame: 9
Processing frame: 10


In [6]:
im.name

'Image1/B2 Region1'